In [2]:
import numpy as np
import pandas as pd
import math
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.cross_validation import train_test_split, KFold, cross_val_score
from sklearn import cross_validation, preprocessing
from sklearn import tree, neighbors
import warnings
warnings.filterwarnings("ignore")

In [3]:
training_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print training_data
print test_data

                  datetime  season  holiday  workingday  weather   temp  \
0      2011-01-01 00:00:00       1        0           0        1   9.84   
1      2011-01-01 01:00:00       1        0           0        1   9.02   
2      2011-01-01 02:00:00       1        0           0        1   9.02   
3      2011-01-01 03:00:00       1        0           0        1   9.84   
4      2011-01-01 04:00:00       1        0           0        1   9.84   
5      2011-01-01 05:00:00       1        0           0        2   9.84   
6      2011-01-01 06:00:00       1        0           0        1   9.02   
7      2011-01-01 07:00:00       1        0           0        1   8.20   
8      2011-01-01 08:00:00       1        0           0        1   9.84   
9      2011-01-01 09:00:00       1        0           0        1  13.12   
10     2011-01-01 10:00:00       1        0           0        1  15.58   
11     2011-01-01 11:00:00       1        0           0        1  14.76   
12     2011-01-01 12:00:0

In [4]:
x_data = []
y_data = []
for i in training_data["count"]:
    y_data.append(i)
y_train = np.array(y_data)
for i in range(len(training_data["count"])):
    data = []
    for j in training_data:
        if j!= "count" and j!= "casual" and j!= "registered":
            data.append(training_data[j][i])
    data.append(int(training_data["datetime"][i][11:13]))
    npdata = np.array(data)
    x_data.append(npdata)
x_train = np.array(x_data)

In [5]:
dates = []
for i in x_train:
    dates.append(i[0])
x_train_copy = []
for i in range(len(x_train)):
    x_train_copy.append(x_train[i][1:])
x_train_copy = preprocessing.scale(x_train_copy)

for i in range(len(x_train)):
    np.insert(x_train[i],0, dates[i])

print dates
print x_train

['2011-01-01 00:00:00', '2011-01-01 01:00:00', '2011-01-01 02:00:00', '2011-01-01 03:00:00', '2011-01-01 04:00:00', '2011-01-01 05:00:00', '2011-01-01 06:00:00', '2011-01-01 07:00:00', '2011-01-01 08:00:00', '2011-01-01 09:00:00', '2011-01-01 10:00:00', '2011-01-01 11:00:00', '2011-01-01 12:00:00', '2011-01-01 13:00:00', '2011-01-01 14:00:00', '2011-01-01 15:00:00', '2011-01-01 16:00:00', '2011-01-01 17:00:00', '2011-01-01 18:00:00', '2011-01-01 19:00:00', '2011-01-01 20:00:00', '2011-01-01 21:00:00', '2011-01-01 22:00:00', '2011-01-01 23:00:00', '2011-01-02 00:00:00', '2011-01-02 01:00:00', '2011-01-02 02:00:00', '2011-01-02 03:00:00', '2011-01-02 04:00:00', '2011-01-02 06:00:00', '2011-01-02 07:00:00', '2011-01-02 08:00:00', '2011-01-02 09:00:00', '2011-01-02 10:00:00', '2011-01-02 11:00:00', '2011-01-02 12:00:00', '2011-01-02 13:00:00', '2011-01-02 14:00:00', '2011-01-02 15:00:00', '2011-01-02 16:00:00', '2011-01-02 17:00:00', '2011-01-02 18:00:00', '2011-01-02 19:00:00', '2011-01-0

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.40, random_state=1338)

In [7]:
def sort_by_date(train, test):
    data_with_date = zip(train, test)
    data_with_date.sort(key=lambda array: array[0][0])
    return zip(*(data_with_date))[0], zip(*(data_with_date))[1]

def cut_date(data):
    new_data = []
    for i in range(len(data)):
        new_data.append(data[i][1:])
    return new_data

def rmsle(y, y_pred):
    for i in range(len(y)):
        if y_pred[i] < 0:
            print "doesn't work"
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]

    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

In [8]:
X_train, Y_train = sort_by_date(X_train, Y_train)
X_test, Y_test = sort_by_date(X_test, Y_test)

In [9]:
#X_train = cut_date(X_train)
#X_test = cut_date(X_test)
count = 0
X_train_copy = X_train
tests_before_test = []
predicted_values = []
KNN_y_train = []
for i in X_test:
    dates_in_lists = set([])
    counts_before_test = []
    data_before_test = []
    total = str(len(X_test))
    for j in range(len(X_train)):
        if (X_train[j][0] not in dates_in_lists) and (X_train[j][0] < i[0]):
            data_before_test.append(X_train[j])
            dates_in_lists.add(X_train[j][0])
            counts_before_test.append(Y_train[j])

    data_before_test = cut_date(data_before_test)
    print str(count+ 1) + "/" + total
    if len(counts_before_test) >= 5:
        est = neighbors.KNeighborsRegressor(n_neighbors=5)
        est.fit(data_before_test, counts_before_test)
        predicted_values.append(est.predict(i[1:]))
        KNN_y_train.append(counts_before_test[len(counts_before_test)-1])
    count += 1

for i in range(len(predicted_values)):
    if predicted_values[i] < 0:
        predicted_values[i] = 0

1/4355
2/4355
3/4355
4/4355
5/4355
6/4355
7/4355
8/4355
9/4355
10/4355
11/4355
12/4355
13/4355
14/4355
15/4355
16/4355
17/4355
18/4355
19/4355
20/4355
21/4355
22/4355
23/4355
24/4355
25/4355
26/4355
27/4355
28/4355
29/4355
30/4355
31/4355
32/4355
33/4355
34/4355
35/4355
36/4355
37/4355
38/4355
39/4355
40/4355
41/4355
42/4355
43/4355
44/4355
45/4355
46/4355
47/4355
48/4355
49/4355
50/4355
51/4355
52/4355
53/4355
54/4355
55/4355
56/4355
57/4355
58/4355
59/4355
60/4355
61/4355
62/4355
63/4355
64/4355
65/4355
66/4355
67/4355
68/4355
69/4355
70/4355
71/4355
72/4355
73/4355
74/4355
75/4355
76/4355
77/4355
78/4355
79/4355
80/4355
81/4355
82/4355
83/4355
84/4355
85/4355
86/4355
87/4355
88/4355
89/4355
90/4355
91/4355
92/4355
93/4355
94/4355
95/4355
96/4355
97/4355
98/4355
99/4355
100/4355
101/4355
102/4355
103/4355
104/4355
105/4355
106/4355
107/4355
108/4355
109/4355
110/4355
111/4355
112/4355
113/4355
114/4355
115/4355
116/4355
117/4355
118/4355
119/4355
120/4355
121/4355
122/4355
123/4355
1

In [10]:
print rmsle(KNN_y_train, predicted_values)

1.10537620056
